In [1]:
import os
os.chdir("/Users/karol/Desktop/Antwerp/ai_project")
import torch
import torch
import torch.nn as nn
from model import MLP1, TwoTower, TwoTowerBasic
from data_reader import load_data, data_preprocessing, load_data_mf, load_customers_articles, customer_buckets
from helper import validate_softmax,  train_softmax, train_two_tower
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from recommenders import recommender_softmax, recommender_two_towers

# Data Preprocessing

### Preprocess and save data

In [2]:
# transactions, articles, customers, article_encodings, customer_encodings = data_preprocessing(feature_generation=False, return_encodings=True, save=True)

### Load preprocessed data and apply one hot encoding for articles and customers

In [2]:
# read data
transactions = pd.read_csv("data/preprocessed/transactions.csv") 
articles = pd.read_csv("data/preprocessed/articles.csv") 
customers = pd.read_csv("data/preprocessed/customers.csv") 

# one hot encoding 
articles = articles.set_index("article_id")
customers = customers.set_index("customer_id")

article_enc = OneHotEncoder(sparse_output=True)
articles = article_enc.fit_transform(articles)

customers_categorical = ["FN",'Active',"club_member_status", "fashion_news_frequency"]
customers_cont = ["age"]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=True), customers_categorical),
        ('cont', 'passthrough', customers_cont)  # 'passthrough' means no transformation for continuous variables
    ],
    remainder='drop'  # Drop any columns not explicitly transformed
)
customers = csr_matrix(preprocessor.fit_transform(customers))

# Softmax DNN

### Assumptions
- The input is the purchase history without last purchase,
- The target is the basket of the last purchase,
- We are interested in whether article was bought not its amount. Therefore, the binary values are allowed.
- In the target basket multiple articles could be bought therefore the sigmoid activation function was used at the last layer. It assumes that decision to buy specific article is independent from other products that customer is buying.


### Load Data & Create Model

In [3]:
train_dataloader, val_dataloader = load_data(transactions, train_test=True, batch_size=1000)

### Create & Train basic Model 

In [7]:
# # create model
# model = MLP1(input_dim=articles.shape[0], output_dim=articles.shape[0])
# # get params
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# save_dir = "AI_project/RQ1/models/MLP1.pt"
# # train
# val_loss_list = train_softmax(model, train_dataloader, val_dataloader, criterion, optimizer, save_dir, num_epochs=5)

  0%|          | 0/5 [11:50<?, ?it/s]


NameError: name 'val_accuracy' is not defined

### Load trained model

In [4]:
MLP1 = torch.load("AI_project/RQ1/models/MLP1.pt")

### Generate recommendations for test data

In [5]:
recommendations, accuracy = recommender_softmax(MLP1, val_dataloader, evaluate=True)
accuracy

/Users/karol/Desktop/Antwerp/ai_project/AI_project/RQ1/data_reader.py:215: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  i = torch.LongTensor(indices)
/Users/karol/Desktop/Antwerp/ai_project/AI_project/RQ1/data_reader.py:219: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:607.)
  return torch.sparse.FloatTensor(i, v, s)
/Users/karol/.pyenv/versions/3.9.17/envs/ai_project/lib/python3.9/site-packages/torch/_tensor_str.py:137: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU.

tensor(0.0028, device='mps:0')

### Ideas for improvements:
- No warm start
- Model should be trained on customers who had at least two purchases
- We take into account baskets which are based on the past 2 years. Maybe we should train model based on customers who bought articles in last month.
- Different way to measure accuracy.
- Customers whou didn't developed any patterns (bought low amount of clothes) should be recommended with current top selling articles.
- Develop more complex (deeper) model.
- More training.
- Distinguish customers who buys specific articles multiple times.
### Potential Issues:
- We need to predict articles for customers who were used to train the model.

**Idea**: recommend things that haven't been bought.



# Matrix Factorization with DNN

In [5]:
buckets = customer_buckets(transactions, train_test=False)
train_dataloader, val_dataloader, test_customers = load_data_mf(transactions, batch_size=1000)

In [6]:
input_article_dim = articles.shape[1]
input_customer_dim = customers.shape[1]
model = TwoTowerBasic(input_article_dim, input_customer_dim, output_dim=3)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
save_dir = "AI_project/RQ1/models/TwoTower1.pt"
train_two_tower(model, customers, articles, buckets, train_dataloader, val_dataloader, criterion, optimizer, save_dir, num_epochs=2)

  5%|▍         | 1369/28589 [00:16<05:26, 83.34it/s]


KeyboardInterrupt: 

In [ ]:
TwoTower1 = torch.load("AI_project/RQ1/models/TwoTower1.pt")

In [ ]:
matrix_full = matrix_representation(transactions, train_test=False)
targets = matrix_full[test_customers]
dataloader_cust, dataloader_art = load_customers_articles(customers, articles, test_customers=test_customers, batch_size=100)


In [ ]:
recommendations, accuracy = recommender_two_towers(TwoTower1, dataloader_cust, dataloader_art, targets, evaluate=True, top_k=5)
accuracy